In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, projectUMAP
from open_spiel.python.examples.ubc_cma import analyze_checkpoint

from auctions.webutils import *



import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Magma256
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

output_notebook()


Loading BokehJS ...

In [10]:
experiment = Experiment.objects.get(name='mar24')
runs = experiment.equilibriumsolverrun_set.all().filter(name__contains='lstm')
experiment = Experiment.objects.get(name='mar22_4')
runs = runs | experiment.equilibriumsolverrun_set.filter(name__contains='lstm')


In [27]:
BASE_GAME = 'large_game_3b'
MODS = ['', '_pricing05', '_hide'] # '_low_reserve'
records = []
for mod in MODS:
    game_name = f'{BASE_GAME}{mod}'
    print(game_name)
    for run in tqdm(runs.filter(game__name=f'{game_name}.json')):
        print(run)
        nc_by_t, checkpoint, approx_nash_conv = find_best_checkpoint(run)
        record = analyze_checkpoint(checkpoint)
        record['approx_nash_conv'] = approx_nash_conv
        record['game'] = game_name
        record['full_name'] = str(run)
        records.append(record)
    print()
df = pd.DataFrame.from_records(records)

large_game_3b


  0%|                                                                                                                                                                               | 0/5 [00:00<?, ?it/s]

large_game_3b-mar17lstm-102


 20%|█████████████████████████████████▍                                                                                                                                     | 1/5 [00:02<00:10,  2.50s/it]

large_game_3b-mar17lstm-101


...

 60%|████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                  | 3/5 [00:07<00:04,  2.40s/it]

large_game_3b_hide-mar17lstm-102


 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                 | 4/5 [00:09<00:02,  2.39s/it]

large_game_3b_hide-mar17lstm-100


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.40s/it]

In [28]:
df

,p0_utility,p0_payment,p1_utility,p1_payment,p2_utility,p2_payment,total_welfare,total_revenue,auction_lengths,common_allocations,approx_nash_conv,game,full_name
0,54.411716,299.052786,45.290427,195.194374,44.656600,129.059001,767.664903,623.306161,8.07355,"[((3, 2, 0), 18431), ((3, 1, 0), 13646), ((2, ...",20.055481,large_game_3b,large_game_3b-mar17lstm-102
1,53.061857,257.590645,53.807459,215.690442,44.941293,133.957708,759.049403,607.238795,7.60950,"[((3, 0, 0), 16924), ((2, 1, 0), 15051), ((1, ...",9.834042,large_game_3b,large_game_3b-mar17lstm-101
2,57.529517,259.787484,47.673981,202.862421,52.972680,137.902320,758.728403,600.552225,7.45474,"[((2, 1, 0), 19025), ((3, 0, 0), 15189), ((1, ...",26.243911,large_game_3b,large_game_3b-mar17lstm-104
3,58.620812,236.914789,61.376084,229.908117,42.457320,135.426480,764.703603,602.249387,7.91764,"[((3, 0, 0), 16777), ((2, 1, 0), 15693), ((1, ...",2.142279,large_game_3b,large_game_3b-mar17lstm-103
4,60.374948,251.612253,54.792549,209.005653,44.770213,138.966188,759.521803,599.584093,7.69340,"[((3, 0, 0), 23598), ((2, 1, 0), 16575), ((1, ...",11.930518,large_game_3b,large_game_3b-mar17lstm-100
5,44.838164,206.184900,59.643019,330.922658,64.174180,110.454598,816.217519,647.562156,8.35799,"[((1, 1, 0), 15462), ((0, 1, 0), 8989), ((2, 1...",0.000000,large_game_3b_pricing05,large_game_3b_pricing05-mar17lstm-101
6,70.925503,223.428442,71.075239,236.993455,66.448881,142.361414,811.232934,602.783310,7.98680,"[((2, 1, 0), 14364), ((1, 2, 0), 13788), ((1, ...",10.414978,large_game_3b_pricing05,large_game_3b_pricing05-mar17lstm-100
7,48.363316,236.359605,53.895953,285.788141,59.811946,128.838241,813.057201,650.985986,8.48682,"[((1, 1, 0), 12151), ((2, 1, 0), 10024), ((2, ...",0.000000,large_game_3b_pricing05,large_game_3b_pricing05-mar17lstm-102
8,48.319670,213.935117,59.149417,323.386024,61.196779,108.017678,814.004684,645.338818,8.32353,"[((1, 1, 0), 13633), ((2, 0, 0), 8970), ((2, 1...",0.000000,large_game_3b_pricing05,large_game_3b_pricing05-mar17lstm-104
9,77.896862,245.729846,67.005025,209.123309,76.930635,142.364823,819.050500,597.217978,7.60088,"[((2, 1, 0), 16770), ((3, 0, 0), 15003), ((1, ...",0.391328,large_game_3b_pricing05,large_game_3b_pricing05-mar17lstm-103


In [29]:
for c in df.columns:
    if c not in ['game', 'approx_nash_conv', 'common_allocations', 'full_name']:
        print(c)
        display(df.groupby('game')[c].describe())

p0_utility


,count,mean,std,min,25%,50%,75%,max
game,,,,,,,,
large_game_3b,5.0,56.799770,3.012699,53.061857,54.411716,57.529517,58.620812,60.374948
large_game_3b_hide,5.0,51.278574,10.947272,31.793245,54.377379,56.267304,56.625223,57.329718
large_game_3b_pricing05,5.0,58.068703,15.188306,44.838164,48.319670,48.363316,70.925503,77.896862


p0_payment


,count,mean,std,min,25%,50%,75%,max
game,,,,,,,,
large_game_3b,5.0,260.991591,23.072360,236.914789,251.612253,257.590645,259.787484,299.052786
large_game_3b_hide,5.0,253.906527,10.057795,243.019522,243.927179,255.633357,263.040383,263.912197
large_game_3b_pricing05,5.0,225.127582,16.105862,206.184900,213.935117,223.428442,236.359605,245.729846


p1_utility


...

,count,mean,std,min,25%,50%,75%,max
game,,,,,,,,
large_game_3b,5.0,761.933623,4.028939,758.728403,759.049403,759.521803,764.703603,767.664903
large_game_3b_hide,5.0,754.277843,6.391778,743.490103,754.030403,756.357803,757.624402,759.886503
large_game_3b_pricing05,5.0,814.712567,3.017935,811.232934,813.057201,814.004684,816.217519,819.050500


total_revenue


,count,mean,std,min,25%,50%,75%,max
game,,,,,,,,
large_game_3b,5.0,606.586132,9.800883,599.584093,600.552225,602.249387,607.238795,623.306161
large_game_3b_hide,5.0,605.140885,15.646556,595.208682,596.782779,600.355078,600.529723,632.828165
large_game_3b_pricing05,5.0,628.777650,26.419968,597.217978,602.783310,645.338818,647.562156,650.985986


auction_lengths


,count,mean,std,min,25%,50%,75%,max
game,,,,,,,,
large_game_3b,5.0,7.749766,0.246429,7.45474,7.60950,7.69340,7.91764,8.07355
large_game_3b_hide,5.0,7.448618,0.213978,7.18870,7.36766,7.40765,7.50829,7.77079
large_game_3b_pricing05,5.0,8.151204,0.358811,7.60088,7.98680,8.32353,8.35799,8.48682
